### pobranie danych źródłowych z portalu IMGW dla wybranego roku, miesiąca i zakresu danych

In [1]:
import requests, zipfile, io

# file = requests.get('https://danepubliczne.imgw.pl/datastore/getfiledown/Arch/Telemetria/Meteo/2024/Meteo_2024-10.zip')
# zip = zipfile.ZipFile(io.BytesIO(file.content))
# zip.extractall("dane_imgw")

## wczytanie danych meteo do struktur danych modułu Pandas

In [2]:
import pandas as pd

files = {
    "B00202A": "dane_imgw/B00202A_2024_10.csv",
    "B00300S": "dane_imgw/B00300S_2024_10.csv",
    "B00305A": "dane_imgw/B00305A_2024_10.csv",
    "B00604S": "dane_imgw/B00604S_2024_10.csv",
    "B00606S": "dane_imgw/B00606S_2024_10.csv",
    "B00608S": "dane_imgw/B00608S_2024_10.csv",
    "B00702A": "dane_imgw/B00702A_2024_10.csv",
    "B00703A": "dane_imgw/B00703A_2024_10.csv",
    "B00714A": "dane_imgw/B00714A_2024_10.csv",
    "B00802A": "dane_imgw/B00802A_2024_10.csv",
}

dataframes = {}
for key, path in files.items():
    df = pd.read_csv(path, header=None, sep=";")
    df.columns = ["kodSH", "parametrSH", "czas", f"{key}"]
    dataframes[key] = df

base_key = "B00202A"
base_df = dataframes[base_key][["kodSH", "czas"]] .drop_duplicates() 

for key, df in dataframes.items():
    if key == base_key:
        continue 
    trimmed = df[["kodSH", "czas", f"{key}"]]
    base_df = pd.merge(base_df, trimmed, on=["kodSH", "czas"], how="left")

base_df 

,kodSH,czas,B00300S,B00305A,B00604S,B00606S,B00608S,B00702A,B00703A,B00714A,B00802A
0,249190090,2024-10-01 00:00,3.29,0.17,NaN,0.0,0.0,0.0,0.9,NaN,92.06
1,249190090,2024-10-01 00:10,3.16,0.11,NaN,NaN,0.0,0.0,0.2,NaN,91.97
2,249190090,2024-10-01 00:20,3.05,0.03,NaN,NaN,0.0,0.0,0.7,NaN,91.98
3,249190090,2024-10-01 00:30,3.01,-0.02,NaN,NaN,0.0,0.2,1.4,NaN,92.45
4,249190090,2024-10-01 00:40,3.07,0.00,NaN,NaN,0.0,0.1,1.5,NaN,92.50
...,...,...,...,...,...,...,...,...,...,...,...
1166100,254150060,2024-10-30 18:10,11.50,11.00,NaN,NaN,0.0,7.0,9.3,NaN,80.90
1166101,254150060,2024-10-30 18:20,11.40,10.60,NaN,NaN,0.0,6.7,9.2,NaN,80.90
1166102,254150060,2024-10-30 18:30,11.40,10.70,NaN,NaN,0.0,6.8,9.4,NaN,81.70
1166103,254150060,2024-10-30 18:40,11.40,10.90,NaN,NaN,0.0,6.2,8.7,NaN,82.00


### wczytanie danych przestrzennych do struktur danych modułu GeoPandas

In [ ]:
import geopandas as gpd

powiaty = gpd.read_file("dane_przestrzenne/powiaty.shp")
wojewodztwa = gpd.read_file("dane_przestrzenne/woj.shp")
effacility = gpd.read_file("dane_przestrzenne/effacility.geojson")

powiaty

### użycie astrala

In [5]:
from astral import LocationInfo
from astral.sun import sun
import datetime

city = LocationInfo("Warsaw", "Poland", "Europe/Warsaw", 52.232222, 21.008333)
s = sun(city.observer, date=datetime.date(2021, 2, 14), tzinfo=city.timezone)
print(s["dawn"], s["sunrise"], s["noon"], s["sunset"], s["dusk"])

2021-02-14 06:17:41.086058+01:00 2021-02-14 06:53:35.458511+01:00 2021-02-14 11:50:08+01:00 2021-02-14 16:47:27.725619+01:00 2021-02-14 17:23:24.874494+01:00
